The goal of this notebook is to reassign segmentation labels based on the objects that they are contained in. 
To do so, a hierarchy of objects must first be defined.
# The hierarchy of objects is defined as follows:
- **Cell**
    - **Nucleus**
    - **Cytoplasm**

The index of a given cytoplasm should be the same as that of cell it came from.
The nucleus index should be the same as that of the cell it came from. 

There will also be rules implemented for sandwiched indexes.
This is when an object was not related properly and was assigned a different index while being surrounded (above and below in the z dimension) by the same object.
Such cases will be assigned the same index as the object that is above and below it.


In [1]:
import argparse
import pathlib
import sys

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import skimage
import skimage.io as io
import tifffile
from cellpose import core, models, utils
from rich.pretty import pprint

sys.path.append("../../utils")
import nviz
from nviz.image_meta import extract_z_slice_number_from_filename, generate_ome_xml
from segmentation_decoupling import euclidian_2D_distance

# check if in a jupyter notebook
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [2]:
if not in_notebook:
    print("Running as script")
    # set up arg parser
    parser = argparse.ArgumentParser(description="Segment the nuclei of a tiff image")

    parser.add_argument(
        "--patient",
        type=str,
        help="The patient ID",
    )

    parser.add_argument(
        "--well_fov",
        type=str,
        help="Path to the input directory containing the tiff images",
    )
    parser.add_argument(
        "--radius_constraint",
        type=int,
        default=10,
        help="The maximum radius of the x-y vector",
    )
    parser.add_argument(
        "--compartment",
        type=str,
        default="none",
        help="The compartment to segment",
    )

    args = parser.parse_args()
    well_fov = args.well_fov
    x_y_vector_radius_max_constaint = args.radius_constraint
    compartment = args.compartment
    patient = args.patient
else:
    print("Running in a notebook")
    well_fov = "C4-2"
    compartment = "nuclei"
    patient = "NF0014"

mask_dir = pathlib.Path(f"../../data/{patient}/processed_data/{well_fov}").resolve()

Running in a notebook


In [3]:
# get the organoid masks
# cell_mask_path = mask_dir / "cell_masks_reconstructed_corrected.tiff"
cell_mask_path = mask_dir / "cell_mask_watershed.tiff"
# cell_mask_output_path = mask_dir / "cell_masks_reassigned.tiff"
nuclei_mask_path = mask_dir / "nuclei_masks_reconstructed.tiff"
cell_mask = io.imread(cell_mask_path)
nuclei_mask = io.imread(nuclei_mask_path)

In [4]:
# get the centroid and bbox of the cell mask
cell_df = pd.DataFrame.from_dict(
    skimage.measure.regionprops_table(
        cell_mask,
        properties=["centroid", "bbox"],
    )
)
cell_df["compartment"] = "cell"
cell_df["label"] = cell_mask[
    cell_df["centroid-0"].astype(int),
    cell_df["centroid-1"].astype(int),
    cell_df["centroid-2"].astype(int),
]
# remove all 0 labels
cell_df = cell_df[cell_df["label"] > 0].reset_index(drop=True)
cell_df["new_label"] = cell_df["label"]

In [5]:
nuclei_df = pd.DataFrame.from_dict(
    skimage.measure.regionprops_table(
        nuclei_mask,
        properties=["centroid", "bbox"],
    )
)
nuclei_df["compartment"] = "nuclei"
nuclei_df["label"] = nuclei_mask[
    nuclei_df["centroid-0"].astype(int),
    nuclei_df["centroid-1"].astype(int),
    nuclei_df["centroid-2"].astype(int),
]
nuclei_df = nuclei_df[nuclei_df["label"] > 0].reset_index(drop=True)

In [6]:
nuclei_df.head()

,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,compartment,label
0,10.934588,746.901225,473.777777,6,687,421,17,808,528,nuclei,14
1,7.888706,224.482524,715.338595,0,165,652,17,283,780,nuclei,24
2,4.147467,253.489444,503.487076,0,195,453,10,313,564,nuclei,34
3,19.420771,424.838544,693.588996,6,369,639,33,474,757,nuclei,39
4,5.122876,694.579159,399.909088,0,649,355,12,745,447,nuclei,46


In [7]:
cell_df.head()

,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,compartment,label,new_label
0,13.130788,748.836667,463.621501,0,597,277,33,897,595,cell,15,15
1,8.603668,204.438405,717.552167,0,78,573,33,325,895,cell,26,26
2,4.749062,230.642543,486.480918,0,89,376,20,354,588,cell,37,37
3,17.425567,436.635295,715.628292,0,361,598,33,533,834,cell,43,43
4,4.753837,691.542028,387.468363,0,581,287,26,828,479,cell,51,51


In [ ]:
def remove_edge_cases(
    mask: np.ndarray,
    border: int = 10,
) -> np.ndarray:
    """
    Remove masks that are image edge cases

    Parameters
    ----------
    mask : np.ndarray
        The mask to process, should be a 3D numpy array
    border : int, optional
        The number of pixels in width to create border to scan for edge cased, by default 10

    Returns
    -------
    np.ndarray
        The mask with edge cases removed
    """

    edge_pixels = np.concatenate(
        [
            #
            mask[
                :, -border:, :
            ].flatten(),  # all of z, last n rows (y), all columns (x) - bottom edge
            mask[
                :, 0:border, :
            ].flatten(),  # all of z, first n rows (y), all columns (x) - top edge
            mask[
                :, :, 0:border:
            ].flatten(),  # all of z, all rows (y), first n columns (x) - left edge
            mask[
                :, :, -border:
            ].flatten(),  # all of z, all rows (y), last n columns (x) - right edge
            # each are the edges stacked for the whole volume -> no need to specify every z slice or 3D edge
        ]
    )
    # get unique edge pixel values
    edge_pixels = np.unique(edge_pixels[edge_pixels > 0])

    for edge_pixel_case in edge_pixels:
        # make the edge cases equal to zero
        mask[mask == edge_pixel_case] = 0

    # return the mask with edge cases removed
    return mask

In [10]:
def centroid_within_bbox_detection(
    centroid: tuple,
    bbox: tuple,
) -> bool:
    """
    Check if the centroid is within the bbox

    Parameters
    ----------
    centroid : tuple
        Centroid of the object in the order of (z, y, x)
        Order of the centroid is important
    bbox : tuple
        Where the bbox is in the order of (z_min, y_min, x_min, z_max, y_max, x_max)
        Order of the bbox is important

    Returns
    -------
    bool
        True if the centroid is within the bbox, False otherwise
    """
    z_min, y_min, x_min, z_max, y_max, x_max = bbox
    z, y, x = centroid
    # check if the centroid is within the bbox
    if (
        z >= z_min
        and z <= z_max
        and y >= y_min
        and y <= y_max
        and x >= x_min
        and x <= x_max
    ):
        return True
    else:
        return False

In [11]:
nuclei_df = nuclei_df.head(10)
cell_df = cell_df.head(10)

In [12]:
# if a centroid of the nuclei is inside the cell mask,
# then make the cell retain the label of the nuclei
for i, row in nuclei_df.iterrows():
    for j, row2 in cell_df.iterrows():
        nuc_contained_in_cell_bool = centroid_within_bbox_detection(
            centroid=(
                row["centroid-0"],
                row["centroid-1"],
                row["centroid-2"],
            ),
            bbox=(
                row2["bbox-0"],
                row2["bbox-1"],
                row2["bbox-2"],
                row2["bbox-3"],
                row2["bbox-4"],
                row2["bbox-5"],
            ),
        )
        if nuc_contained_in_cell_bool:
            print(
                f"x-min: {row2['bbox-0']} **coordinate: {row['centroid-0']}** x-max: {row2['bbox-3']}"
            )
            print(
                f"y-min: {row2['bbox-1']} **coordinate: {row['centroid-1']}** y-max: {row2['bbox-4']}"
            )
            print(
                f"z-min: {row2['bbox-2']} **coordinate: {row['centroid-2']}** z-max: {row2['bbox-5']}"
            )
            # if the centroid of the nuclei is within the cell mask,
            # then make the cell retain the label of the nuclei
            cell_df.at[j, "new_label"] = row["label"]
            # print(f"Cell {row2['label']} contains nuclei {row['label']}")
            break
        else:
            # print(f"Cell {row2['label']} does not contain nuclei {row['label']}")
            pass

x-min: 0 **coordinate: 10.934588254181676** x-max: 33
y-min: 597 **coordinate: 746.9012251532696** y-max: 897
z-min: 277 **coordinate: 473.77777666288716** z-max: 595
x-min: 0 **coordinate: 7.8887062630707145** x-max: 33
y-min: 78 **coordinate: 224.48252441465584** y-max: 325
z-min: 573 **coordinate: 715.3385946623519** z-max: 895
x-min: 0 **coordinate: 4.147466638248493** x-max: 20
y-min: 89 **coordinate: 253.48944383266434** y-max: 354
z-min: 376 **coordinate: 503.4870756515458** z-max: 588
x-min: 0 **coordinate: 19.4207714907093** x-max: 33
y-min: 361 **coordinate: 424.8385440115991** y-max: 533
z-min: 598 **coordinate: 693.588996146362** z-max: 834
x-min: 0 **coordinate: 5.122876384966326** x-max: 33
y-min: 597 **coordinate: 694.5791585198059** y-max: 897
z-min: 277 **coordinate: 399.9090882757622** z-max: 595
x-min: 0 **coordinate: 3.5257416742417984** x-max: 33
y-min: 333 **coordinate: 363.4036856458144** y-max: 398
z-min: 134 **coordinate: 177.25535079745475** z-max: 209
x-min: 

In [13]:
def mask_label_reassignment(
    mask_df: pd.DataFrame,
    mask_input: np.ndarray,
) -> np.ndarray:
    """
    Reassign the labels of the mask based on the mask_df

    Parameters
    ----------
    mask_df : pd.DataFrame
        DataFrame containing the labels and centroids of the mask
    mask_input : np.ndarray
        The input mask to reassign the labels to

    Returns
    -------
    np.ndarray
        The mask with reassigned labels
    """
    for i, row in mask_df.iterrows():
        if row["label"] == row["new_label"]:
            # if the label is already the new label, skip
            continue
        mask_input[mask_input == row["label"]] = row["new_label"]
    return mask_input

In [14]:
cell_mask = mask_label_reassignment(
    mask_df=cell_df,
    mask_input=cell_mask,
)
tifffile.imwrite(
    cell_mask_output_path,
    cell_mask,
)

NameError: name 'cell_mask_output_path' is not defined